In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import time
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

pd.set_option('display.max_columns', None)
# Import our input dataset
dogs = pd.read_csv('../Resources/Shelter_adoption_demograpics.csv')
dogs.head()

,index_id_intake,animal_id_intake,breed_category,breed_consolidated,breed_group,breed_contains_pitbull,color_consolidated,intake_type,intake_condition,sex_upon_intake_subtype,age_intake(days),age_intake(years),age_range_intake,intake_date,intake_month,intake_day1,intake_year,intake_numeric_weekday,intake_hour,outcome_date,outcome_month,outcome_day,outcome_year,outcome_weekday,age_outcome(days),age_upon_outcome(years),age_range_outcome,days_in_shelter,outcome,zipcode,population,population_density,land_area_in_sqmi,housing_units,median_household_income,Male_Population,Female_Population,Less_than_high_school,High_School_Graduate,Associates_degree,Bachelors_degree,Masters_degree,Professional_school_degree,Doctorate_degree,Race_white,Race_black_African_American,Race_American_Indian_Alaskan_native,Race_Asian,Race_Native_Hawaiian_Pacific_Islander,Race-Other,Race_Two_or_more
0,142136,A362707,Mix,retriever,sporting,N,Black,Owner Surrender,Sick,Female,4015,11.0,"(10.0, 12.5]",1/27/2016,1,27,2016,2,12,1/10/2017,1.0,10.0,2017.0,Tuesday,4667.0,12.79,"(12.5, 15.0]",348,Success,78725,6083,346,17.57,1978,52381,2988,3095,1890,2481,120,597,127,0,15,2924,1838,96,55,13,970,187
1,130786,A396810,Mix,retriever,sporting,N,Black,Stray,Injured,Female,4745,13.0,"(12.5, 15.0]",8/22/2016,8,22,2016,0,16,12/23/2016,12.0,23.0,2016.0,Friday,4912.0,13.46,"(12.5, 15.0]",122,Success,78724,21696,889,24.40,6138,38479,11181,10515,4341,5270,683,915,245,18,71,8339,6167,239,100,6,6165,680
2,123222,A428783,Purebred,herding,working-herding,N,Other,Public Assist,Normal,Female,2920,8.0,"(7.5, 10.0]",1/31/2014,1,31,2014,4,16,2/13/2014,2.0,13.0,2014.0,Thursday,3023.0,8.28,"(7.5, 10.0]",13,Success,78758,44072,4747,9.28,19577,43537,23014,21058,6267,12480,1690,6744,2167,169,200,25073,4810,467,2911,29,9196,1586
3,58675,A442798,Mix,terrier,terrier,N,White/Other,Public Assist,Normal,Female,3285,9.0,"(7.5, 10.0]",4/10/2015,4,10,2015,4,16,4/29/2015,4.0,29.0,2015.0,Wednesday,3485.0,9.55,"(7.5, 10.0]",18,Success,78752,18064,5403,3.34,7944,34716,9778,8286,3976,4429,493,2095,807,88,120,10787,2124,334,333,10,3781,695
4,76282,A443790,Mix,herding,working-herding,N,Black/Other,Stray,Normal,Male,3285,9.0,"(7.5, 10.0]",9/3/2014,9,3,2014,2,17,9/13/2014,9.0,13.0,2014.0,Saturday,3312.0,9.07,"(7.5, 10.0]",10,Success,78745,55614,4166,13.35,25749,50672,27813,27801,5393,18935,2426,10258,3204,587,359,40450,2596,601,1133,32,8730,2072


In [2]:
# Create new DF with animal_ids
#dogs.set_index("index_id_intake", inplace = True)

animal_id = pd.DataFrame().assign(animal_id_intake = dogs["animal_id_intake"])
dogs.head()

,index_id_intake,animal_id_intake,breed_category,breed_consolidated,breed_group,breed_contains_pitbull,color_consolidated,intake_type,intake_condition,sex_upon_intake_subtype,age_intake(days),age_intake(years),age_range_intake,intake_date,intake_month,intake_day1,intake_year,intake_numeric_weekday,intake_hour,outcome_date,outcome_month,outcome_day,outcome_year,outcome_weekday,age_outcome(days),age_upon_outcome(years),age_range_outcome,days_in_shelter,outcome,zipcode,population,population_density,land_area_in_sqmi,housing_units,median_household_income,Male_Population,Female_Population,Less_than_high_school,High_School_Graduate,Associates_degree,Bachelors_degree,Masters_degree,Professional_school_degree,Doctorate_degree,Race_white,Race_black_African_American,Race_American_Indian_Alaskan_native,Race_Asian,Race_Native_Hawaiian_Pacific_Islander,Race-Other,Race_Two_or_more
0,142136,A362707,Mix,retriever,sporting,N,Black,Owner Surrender,Sick,Female,4015,11.0,"(10.0, 12.5]",1/27/2016,1,27,2016,2,12,1/10/2017,1.0,10.0,2017.0,Tuesday,4667.0,12.79,"(12.5, 15.0]",348,Success,78725,6083,346,17.57,1978,52381,2988,3095,1890,2481,120,597,127,0,15,2924,1838,96,55,13,970,187
1,130786,A396810,Mix,retriever,sporting,N,Black,Stray,Injured,Female,4745,13.0,"(12.5, 15.0]",8/22/2016,8,22,2016,0,16,12/23/2016,12.0,23.0,2016.0,Friday,4912.0,13.46,"(12.5, 15.0]",122,Success,78724,21696,889,24.40,6138,38479,11181,10515,4341,5270,683,915,245,18,71,8339,6167,239,100,6,6165,680
2,123222,A428783,Purebred,herding,working-herding,N,Other,Public Assist,Normal,Female,2920,8.0,"(7.5, 10.0]",1/31/2014,1,31,2014,4,16,2/13/2014,2.0,13.0,2014.0,Thursday,3023.0,8.28,"(7.5, 10.0]",13,Success,78758,44072,4747,9.28,19577,43537,23014,21058,6267,12480,1690,6744,2167,169,200,25073,4810,467,2911,29,9196,1586
3,58675,A442798,Mix,terrier,terrier,N,White/Other,Public Assist,Normal,Female,3285,9.0,"(7.5, 10.0]",4/10/2015,4,10,2015,4,16,4/29/2015,4.0,29.0,2015.0,Wednesday,3485.0,9.55,"(7.5, 10.0]",18,Success,78752,18064,5403,3.34,7944,34716,9778,8286,3976,4429,493,2095,807,88,120,10787,2124,334,333,10,3781,695
4,76282,A443790,Mix,herding,working-herding,N,Black/Other,Stray,Normal,Male,3285,9.0,"(7.5, 10.0]",9/3/2014,9,3,2014,2,17,9/13/2014,9.0,13.0,2014.0,Saturday,3312.0,9.07,"(7.5, 10.0]",10,Success,78745,55614,4166,13.35,25749,50672,27813,27801,5393,18935,2426,10258,3204,587,359,40450,2596,601,1133,32,8730,2072


In [3]:
dogs.drop(["index_id_intake", "animal_id_intake","intake_date", "outcome_date", "outcome_month", "outcome_day", "outcome_year", "outcome_weekday",
          "age_outcome(days)", "age_upon_outcome(years)", "outcome_weekday", "age_outcome(days)", "age_upon_outcome(years)",
           "age_range_outcome"], axis = 1, inplace = True)

In [4]:
# Generate our categorical variable lists
dogs_cat = dogs.dtypes[dogs.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
dogs[dogs_cat].nunique()

breed_category              2
breed_consolidated         12
breed_group                11
breed_contains_pitbull      2
color_consolidated         14
intake_type                 4
intake_condition           10
sex_upon_intake_subtype     3
age_range_intake            8
outcome                     2
dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(dogs[dogs_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(dogs_cat)
encode_df.head(-10)

,breed_category_Mix,breed_category_Purebred,breed_consolidated_Malinois/German Shepherd,breed_consolidated_chihuahua,breed_consolidated_guardian,breed_consolidated_herding,breed_consolidated_hound,breed_consolidated_nonsporting,breed_consolidated_pit bull,breed_consolidated_pointer,breed_consolidated_retriever,breed_consolidated_terrier,breed_consolidated_toy,breed_consolidated_working,breed_consolidated_nan,breed_group_Hound,breed_group_Nonsporting,breed_group_Pit bull terrier,breed_group_hound,breed_group_nonsporting,breed_group_pit bull,breed_group_sporting,breed_group_terrier,breed_group_toy,breed_group_working,breed_group_working-herding,breed_group_nan,breed_contains_pitbull_N,breed_contains_pitbull_Y,color_consolidated_Black,color_consolidated_Black/Other,color_consolidated_Black/White,color_consolidated_Brindle,color_consolidated_Brown,color_consolidated_Brown/Other,color_consolidated_Brown/White,color_consolidated_Other,color_consolidated_Red/Other,color_consolidated_Tan/Tan Mix,color_consolidated_Tan/White,color_consolidated_White,color_consolidated_White/Black or Brown,color_consolidated_White/Other,intake_type_Abandoned,intake_type_Owner Surrender,intake_type_Public Assist,intake_type_Stray,intake_condition_Aged,intake_condition_Behavior,intake_condition_Injured,intake_condition_Medical,intake_condition_Neonatal,intake_condition_Normal,intake_condition_Nursing,intake_condition_Other,intake_condition_Pregnant,intake_condition_Sick,sex_upon_intake_subtype_Female,sex_upon_intake_subtype_Male,sex_upon_intake_subtype_Unknown,"age_range_intake_(-0.001, 2.5]","age_range_intake_(10.0, 12.5]","age_range_intake_(12.5, 15.0]","age_range_intake_(15.0, 17.5]","age_range_intake_(17.5, 20.0]","age_range_intake_(2.5, 5.0]","age_range_intake_(5.0, 7.5]","age_range_intake_(7.5, 10.0]",outcome_Fail,outcome_Success
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8130,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [6]:
# Merge one-hot encoded features and drop the originals
dogs = dogs.merge(encode_df,left_index=True, right_index=True)
dogs = dogs.drop(dogs_cat,1)
dogs.head(-10)

C:\Users\arroc\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,age_intake(days),age_intake(years),intake_month,intake_day1,intake_year,intake_numeric_weekday,intake_hour,days_in_shelter,zipcode,population,population_density,land_area_in_sqmi,housing_units,median_household_income,Male_Population,Female_Population,Less_than_high_school,High_School_Graduate,Associates_degree,Bachelors_degree,Masters_degree,Professional_school_degree,Doctorate_degree,Race_white,Race_black_African_American,Race_American_Indian_Alaskan_native,Race_Asian,Race_Native_Hawaiian_Pacific_Islander,Race-Other,Race_Two_or_more,breed_category_Mix,breed_category_Purebred,breed_consolidated_Malinois/German Shepherd,breed_consolidated_chihuahua,breed_consolidated_guardian,breed_consolidated_herding,breed_consolidated_hound,breed_consolidated_nonsporting,breed_consolidated_pit bull,breed_consolidated_pointer,breed_consolidated_retriever,breed_consolidated_terrier,breed_consolidated_toy,breed_consolidated_working,breed_consolidated_nan,breed_group_Hound,breed_group_Nonsporting,breed_group_Pit bull terrier,breed_group_hound,breed_group_nonsporting,breed_group_pit bull,breed_group_sporting,breed_group_terrier,breed_group_toy,breed_group_working,breed_group_working-herding,breed_group_nan,breed_contains_pitbull_N,breed_contains_pitbull_Y,color_consolidated_Black,color_consolidated_Black/Other,color_consolidated_Black/White,color_consolidated_Brindle,color_consolidated_Brown,color_consolidated_Brown/Other,color_consolidated_Brown/White,color_consolidated_Other,color_consolidated_Red/Other,color_consolidated_Tan/Tan Mix,color_consolidated_Tan/White,color_consolidated_White,color_consolidated_White/Black or Brown,color_consolidated_White/Other,intake_type_Abandoned,intake_type_Owner Surrender,intake_type_Public Assist,intake_type_Stray,intake_condition_Aged,intake_condition_Behavior,intake_condition_Injured,intake_condition_Medical,intake_condition_Neonatal,intake_condition_Normal,intake_condition_Nursing,intake_condition_Other,intake_condition_Pregnant,intake_condition_Sick,sex_upon_intake_subtype_Female,sex_upon_intake_subtype_Male,sex_upon_intake_subtype_Unknown,"age_range_intake_(-0.001, 2.5]","age_range_intake_(10.0, 12.5]","age_range_intake_(12.5, 15.0]","age_range_intake_(15.0, 17.5]","age_range_intake_(17.5, 20.0]","age_range_intake_(2.5, 5.0]","age_range_intake_(5.0, 7.5]","age_range_intake_(7.5, 10.0]",outcome_Fail,outcome_Success
0,4015,11.00,1,27,2016,2,12,348,78725,6083,346,17.57,1978,52381,2988,3095,1890,2481,120,597,127,0,15,2924,1838,96,55,13,970,187,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,4745,13.00,8,22,2016,0,16,122,78724,21696,889,24.40,6138,38479,11181,10515,4341,5270,683,915,245,18,71,8339,6167,239,100,6,6165,680,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2920,8.00,1,31,2014,4,16,13,78758,44072,4747,9.28,19577,43537,23014,21058,6267,12480,1690,6744,2167,169,200,25073,4810,467,2911,29,9196,1586,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,3285,9.00,4,10,2015,4,16,18,78752,18064,5403,3.34,7944,34716,9778,8286,3976,4429,493,2095,807,88,120,10787,2124,334,333,10,3781,695,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.

In [8]:
# Split our preprocessed data into our features and target arrays. Split before using standard scaler so that testing
# data isn't influencing the standardization
y = dogs["outcome_Success"].values
X = dogs.drop(["outcome_Success","outcome_Fail"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\arroc\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [9]:
feature_names = dogs.drop(["outcome_Success","outcome_Fail"],1).columns

C:\Users\arroc\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [11]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  25
hidden_nodes_layer2 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                2475      
                                                                 
 dense_1 (Dense)             (None, 25)                650       
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 3,151
Trainable params: 3,151
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
191/191 [==============================] - 1s 1ms/step - loss: 0.4785 - accuracy: 0.7731
Epoch 2/100
191/191 [==============================] - 0s 1ms/step - loss: 0.1248 - accuracy: 0.9735
Epoch 3/100
191/191 [==============================] - 0s 1ms/step - loss: 0.0370 - accuracy: 0.9939
Epoch 4/100
191/191 [==============================] - 0s 1ms/step - loss: 0.0223 - accuracy: 0.9956
Epoch 5/100
191/191 [==============================] - 0s 1ms/step - loss: 0.0174 - accuracy: 0.9967
Epoch 6/100
191/191 [==============================] - 0s 2ms/step - loss: 0.0142 - accuracy: 0.9971
Epoch 7/100
191/191 [==============================] - 0s 1ms/step - loss: 0.0122 - accuracy: 0.9977
Epoch 8/100
191/191 [==============================] - 0s 1ms/step - loss: 0.0094 - accuracy: 0.9980
Epoch 9/100
191/191 [==============================] - 0s 1ms/step - loss: 0.0087 - accuracy: 0.9980
Epoch 10/100
191/191 [==============================] - 0s 1ms/step - loss: 0.0073 - accura

191/191 [==============================] - 0s 1ms/step - loss: 5.2843e-05 - accuracy: 1.0000
Epoch 80/100
191/191 [==============================] - 0s 1ms/step - loss: 4.7074e-05 - accuracy: 1.0000
Epoch 81/100
191/191 [==============================] - 0s 1ms/step - loss: 4.1795e-05 - accuracy: 1.0000
Epoch 82/100
191/191 [==============================] - 0s 1ms/step - loss: 3.9263e-05 - accuracy: 1.0000
Epoch 83/100
191/191 [==============================] - 0s 1ms/step - loss: 3.6250e-05 - accuracy: 1.0000
Epoch 84/100
191/191 [==============================] - 0s 1ms/step - loss: 3.4047e-05 - accuracy: 1.0000
Epoch 85/100
191/191 [==============================] - 0s 1ms/step - loss: 3.2068e-05 - accuracy: 1.0000
Epoch 86/100
191/191 [==============================] - 0s 1ms/step - loss: 2.9918e-05 - accuracy: 1.0000
Epoch 87/100
191/191 [==============================] - 0s 1ms/step - loss: 2.7391e-05 - accuracy: 1.0000
Epoch 88/100
191/191 [==============================] - 0s 

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

64/64 - 0s - loss: 0.1169 - accuracy: 0.9907 - 76ms/epoch - 1ms/step
Loss: 0.11685849726200104, Accuracy: 0.990667998790741


## Logistic Regression Model with Data

In [16]:
# Define the logistic regression model
from sklearn.linear_model import LogisticRegression
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.680


C:\Users\arroc\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [17]:
print(list(zip(log_classifier.coef_, feature_names)))

[(array([-2.73306580e-04, -7.57751277e-07,  2.89435760e-05, -4.57517442e-05,
       -6.17525222e-05,  3.93729291e-06,  3.54459456e-03,  3.06316201e-03,
        2.22249643e-06, -2.13591350e-04,  1.03109419e-04,  1.31704610e-05,
        1.66785161e-04,  1.10396262e-05, -4.20485272e-04,  2.06893922e-04,
       -1.28334392e-04,  2.90316608e-05,  1.83487906e-04, -2.85477346e-04,
        3.40594249e-04, -5.40196041e-04,  5.96801357e-06,  3.01901993e-04,
        3.74732194e-04, -6.11081154e-04,  2.52808377e-04, -7.03307628e-06,
        4.92401020e-04, -1.01732070e-03,  1.09301049e-05, -1.09315594e-05,
        6.75471167e-06,  1.60270704e-06, -3.71810231e-07,  1.11898127e-05,
        2.05349475e-07, -2.56133177e-06, -2.46574655e-07,  7.17678058e-07,
       -9.22220488e-07, -5.33257612e-07, -1.49197933e-05, -9.16725340e-07,
        0.00000000e+00,  2.52176294e-07, -3.00548119e-08,  1.09040850e-06,
       -4.34479382e-08, -2.51729255e-06, -6.69910791e-06, -2.07921312e-07,
        4.81488273e-06,

In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  53,  621],
       [  31, 1331]], dtype=int64)

## Random Forest Model with Data

In [19]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.996


In [ ]:
from sklearn.svm import SVC

# Create the SVM model
svm = SVC(kernel="sigmoid")

In [ ]:
# Train the model
svm.fit(X_train, y_train)